In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample


In [2]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [3]:
def createData():
    df = readCsv('1_Cov.csv')
    meanPositive = readCsv('1_m_1.csv').dropna(axis=1, how='all').as_matrix()
    meanNegative = readCsv('1_m_0.csv').dropna(axis=1, how='all').as_matrix()
    meanPositive = meanPositive[0,:]
    meanNegative = meanNegative[0,:]
    
    cov = df.dropna(axis=1, how='all').as_matrix()
    
    distributionPositive = np.random.multivariate_normal(meanPositive, cov, 2000)
    distributionNegative = np.random.multivariate_normal(meanNegative, cov, 2000)
    
    distributionPositive = np.c_[distributionPositive, np.ones(distributionPositive.shape[0])]
    distributionNegative = np.c_[distributionNegative, np.zeros(distributionNegative.shape[0])]
#     print('Positive', distributionPositive)
#     print('Negative', distributionNegative)
    distribution = np.concatenate((distributionPositive, distributionNegative), axis=0)
    np.random.shuffle(distribution)
    distributionTest = distribution[0:1200]
    distributionTrain = distribution[1200:] 
    print(distribution)
    print(distributionTest)
    print(distributionTrain)
    print('original', distribution.shape)
    print('train', distributionTrain.shape)
    print('test', distributionTest.shape)
    return (distribution, distributionTest, distributionTrain)


In [4]:
def toCsv(fileName, npArray):
    
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')


In [5]:
def initializeDS1():
    fullDistribution, distributionTest, distributionTrain = createData()
    toCsv('1_train.csv', distributionTrain)
    toCsv('1_test.csv', distributionTest)
    toCsv('1.csv', fullDistribution)
initializeDS1()


[[-1.96813941 -1.36731598  0.8709495  ..., -1.7888388  -0.32786317  1.        ]
 [ 4.59363397  5.02203898  4.48394658 ...,  5.24934967  2.85462927  1.        ]
 [ 3.50865605  3.51813678  7.26843244 ...,  1.51237812  2.60544689  0.        ]
 ..., 
 [-3.4661588  -2.67880532 -4.42310559 ..., -2.27219633 -2.33953776  0.        ]
 [-1.86869786 -0.85671284 -2.35203223 ..., -2.59177284 -1.07752331  0.        ]
 [-3.86179705 -0.61350199 -1.78123321 ..., -3.60412378  0.51086162  0.        ]]
[[-1.96813941 -1.36731598  0.8709495  ..., -1.7888388  -0.32786317  1.        ]
 [ 4.59363397  5.02203898  4.48394658 ...,  5.24934967  2.85462927  1.        ]
 [ 3.50865605  3.51813678  7.26843244 ...,  1.51237812  2.60544689  0.        ]
 ..., 
 [ 1.56463803 -1.69328951 -1.34517965 ...,  1.18924525  3.01797646  1.        ]
 [ 1.32245224  1.2956472   4.91379902 ...,  1.38080925  3.46909986  0.        ]
 [-0.03891431  2.73332995  1.93012171 ...,  3.12014719  1.89583549  0.        ]]
[[ 0.18969063  1.4326592

In [6]:
def visualizeStuff():
    mean = [10, 10, 10]
    cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]] 
    x, y, z = np.random.multivariate_normal(mean, cov, 10000).T

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(111, projection='3d')

    plt.scatter(x, y, z, c='red')
    plt.axis('equal')
    plt.show()
# visualizeStuff()

In [34]:
def runLDA():
    dfTrain = readCsv('1_train.csv')
    N1, N2, pi, S, U1, U2 = np.zeros(8)
    Y = dfTrain[20].as_matrix()
    dfTrain = dfTrain.iloc[:,0:20]
    
    S1 = np.zeros((20,20))
    S2 = np.zeros((20,20))
    N1 = np.count_nonzero(Y)
    N2 = 2800 - N1
    pi = N1/(N1+N2)
    
    for i in range(0, len(dfTrain)):
        U1 = U1 + Y[i]*dfTrain.iloc[i,:].as_matrix()
        U2 = U2 + (1-Y[i])*dfTrain.iloc[i,:].as_matrix()
    U1 = U1/N1
    U2 = U2/N2
    
    print(U1)
    print(U2)
    
    for i in range(0)

#     print(dfTrain)
    test = dfTrain.iloc[0,:].as_matrix()
#     print(test)
#     print(test.transpose())
#     print(np.dot(test,test))
#     print(np.outer(test, test).size)
#     for i in range(0, len(dfTrain)):
#         print(dfTrain.iloc[i,20])
#         U1 += dfTrain.iloc[i,20]*
runLDA()


[ 2.0481571   2.02347394  2.05037788  2.04524597  2.10530863  2.02750125
  2.06749715  2.05607714  2.00353346  1.96973319  2.06789442  2.04956393
  2.03209717  2.09537586  2.06822274  2.03197757  2.06102946  2.01914767
  2.13865579  2.05738195]
[ 1.34235165  1.27590385  1.2931513   1.33289967  1.34141175  1.28238359
  1.40182503  1.35265194  1.27748009  1.25367154  1.29213937  1.36513627
  1.33504341  1.28175396  1.2678472   1.25515131  1.27928885  1.2649528
  1.35729231  1.27362941]
